In [ ]:
import satellite_images_nso.api.nso_georegion as nso
import satellite_images_nso.api.sat_manipulator as sat_manipulator
import matplotlib.pyplot as plt
import zipfile
import os
import rasterio

In [ ]:
# Inloggegevens voor nso dataportaal
user_name = "jelledegen"
user_password = "niqzen-zoVmep-kyhju6"
path_geojson = "../data/alkmaar_single_polygon.geojson"
output_folder = "../output/"

# The first parameter is the path to the geojson, the second the map where the cropped satellite data will be downloaded, the third is your NSO username and the last your NSO password.
georegion = nso.nso_georegion(
    path_to_geojson=path_geojson,
    output_folder=output_folder,
    username=user_name,
    password=user_password,
)


In [ ]:
# This method fetches all the download links with all the satellite images the NSO has which contain the region in the given geojson.
# Max_diff parameters represents the amount of percentage the selected region has to be in the satellite image.
# So 1 is the the selected region has to be fully in the satellite images while 0.7 donates only 70% of the selected region is in the
links = georegion.retrieve_download_links(
    max_diff=0.5, start_date="2022-01-01", end_date="2022-04-01"
)
print(f"Aantal gevonden links: {len(links)}")

In [ ]:
# This example filters out only 50 cm RGB Infrared Superview satellite imagery in the summer from all the links
season = "Summer"
links_group = []
for link in links:
    if "SV" in link and "50cm" in link and "RGBI" in link:
        if (
            sat_manipulator.get_season_for_month(
                int(link.split("/")[len(link.split("/")) - 1][4:6])
            )[0]
            == season
        ):
            links_group.append(link)

# Downloads a satellite image from the NSO, makes a crop out of it so it fits the geojson region and calculates the NVDI index.
# The output will stored in the output folder.
# The parameters are : link, delete_zip_file = False, delete_source_files = True,  plot=True, in_image_cloud_percentage = False,  add_ndvi_band = False, add_height_band = False
# description of these parameters can be found in the code.
georegion.execute_link(links_group[0], add_ndvi_band=True)


zip_path = "../output/deze.zip"
extract_to = "../output/unzipped/"

# Maak map aan als die nog niet bestaat
os.makedirs(extract_to, exist_ok=True)

# Unzippen
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_to)


In [ ]:
tif_path = "../output/unzipped/20220719_111417_SV1-03_SV_RD_11bit_RGBI_50cm_Alkmaar.tif"

# Checken welke bands NIR en red zijn
with rasterio.open(tif_path) as src:
    print(f"Aantal banden: {src.count}")
    print(f"Band omschrijvingen: {src.descriptions}")  # soms leeg
    for i in range(1, src.count + 1):
        band = src.read(i)
        print(f"Band {i} shape: {band.shape}, min: {band.min()}, max: {band.max()}")

with rasterio.open(tif_path) as src:
    red = src.read(3).astype(float)
    nir = src.read(4).astype(float)

# NDVI berekenen
ndvi = (nir - red) / (nir + red + 1e-10)

# plotten van NDVI map
plt.figure(figsize=(10, 10))
plt.imshow(ndvi, cmap="RdYlGn")
plt.colorbar(label="NDVI waarde")
plt.title("NDVI beeld")
plt.axis("off")
plt.show()
